## What are graphs?

Tensorflow 2.0 runs in two modes:
* **eagrly:** normal Python execution
* **graphs:** portable (no Python interpreter needed), offer speed improvements

graphs are made of *`tf.Operation`* and *`tf.Tensor`*

## How is a graph created?
using `@tf.function` or calling `tf.function()`

a function is transformed into a PolymorphicFunction

* `tf.function` is used to make graphs out of programs, for better performance and deployability
* `tf.function` creates Python-indpendent dataflow graphs
* `tf.function` works best with tf ops and NumPy, might have problems with Python native commands

In [1]:
import tensorflow as tf

In [2]:
# Define a Python function.
def a_regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

In [3]:
# Python function --> PolymorphicFunction (function that uses grapsh)
# The Python type of `a_function_that_uses_a_graph` will now be a
# `PolymorphicFunction`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

In [4]:
# try execution w/ and w/out graph
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
orig_value, tf_function_value

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


(array([[12.]], dtype=float32), array([[12.]], dtype=float32))

In [ ]:
**Note:** `tf.function` applies to all the functions called inside it too.

In [3]:
def inner_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

# Using the `tf.function` decorator makes `outer_function` into a
# `PolymorphicFunction`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

## But how is the graph really created?!

* tf operations are esily trasformed
* Python code is converted to graph-generating code throug the **Autograph library (`tf.autograph`)**

Here's a graph example of the Relu function
* use `tf.autograph.to_code(func)` to view the function
* use `.get_concrete_function(<func input>).graph.as_graph_def()` to view the graph itself

In [7]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0

# to `PolymorphicFunction`
tf_simple_relu = tf.function(simple_relu)

# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

ConversionError: converting <function simple_relu at 0x7fbf5b25e680>: AttributeError: 'arguments' object has no attribute 'posonlyargs'

In [8]:
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


OperatorNotAllowedInGraphError: Using a symbolic `tf.Tensor` as a Python `bool` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.